# EventStoreDB and Python: Getting Started

This Jupyter notebook will guide you through using the EventStoreDB Python client to build a simple eCommerce application. We will start by appending and reading events.

Note that this notebook assume you are running EventStoreDB on port 2113. Look here https://developers.eventstore.com/server/v20.10/installation.html#quick-start if you need to quickly install and run EventStoreDB on your desktop.

## Setting up the Environment

First, we need to install the necessary packages and set up our environment. The EventStoreDB client can be installed via pip:


In [1]:
!pip install esdbclient

     ---------------------------------------- 0.0/102.5 kB ? eta -:--:--
     -------------------------------------- 102.5/102.5 kB 3.0 MB/s eta 0:00:00
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
  Using cached protobuf-4.23.2-cp310-abi3-win_amd64.whl (422 kB)
  Using cached grpcio-1.54.2-cp310-cp310-win_amd64.whl (4.1 MB)


Now, let's import the necessary modules:

In [2]:
import uuid
from esdbclient import EventStoreDBClient, NewEvent, StreamState

## Connecting to EventStoreDB

We'll start by establishing a connection to our EventStoreDB instance. For this example, we'll use an insecure server running locally on port 2113:


In [4]:
client = EventStoreDBClient(uri="esdb://localhost:2113?Tls=false")

## Appending Events

In an eCommerce application, an example of an event would be creating an order. Let's simulate this by appending an `OrderCreated` event to a stream:


In [10]:

stream_name = str(uuid.uuid4())
event = NewEvent(
    type='OrderCreated',
    data=b'{"order_id": "1234", "product_id": "5678", "quantity": 2}'
)
commit_position = client.append_to_stream(
    stream_name=stream_name, 
    current_version=StreamState.NO_STREAM, 
    events=[event]
)


In the above code, we create a unique stream name using a UUID, then create a new `OrderCreated` event with some sample data. We then append this event to the stream.

The `append_to_stream()` method returns a commit position, which is the overall position in the database of the last new event recorded by this operation.


## Reading Events

Now that we've recorded an event, let's read it back:


In [11]:
read_response = client.read_stream(stream_name)

recorded_events = tuple(read_response)

print(recorded_events)

(RecordedEvent(type='OrderCreated', data=b'{"order_id": "1234", "product_id": "5678", "quantity": 2}', metadata=b'', content_type='application/json', id=UUID('b0ae6a7d-a45b-4534-836b-d7e2a734ae83'), stream_name='c17ac815-2bf3-433e-bada-62a0a5933196', stream_position=0, commit_position=6102, retry_count=None),)


In this code, we call `read_stream()` to get all the recorded events in the stream. The method returns a "read response" iterator, which we convert to a tuple to easily view the events. We then print out the recorded events.

This concludes our introduction to appending and reading events with EventStoreDB and Python. In the next sections, we will explore more complex scenarios involving business rules and integration events.